In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score

sys.path.append(os.path.abspath('..'))
from src.utils.seed import set_seed
from src.utils.config import Config
from src.preprocessing import get_data_loaders
from src.ensemble import StackingEnsemble
from src.evaluation import calculate_metrics

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

c:\Users\surya\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


In [2]:
config = Config('../configs/config.yaml')
train_loader, val_loader, test_loader, input_dim = get_data_loaders(config)

X_train_list = []
y_train_list = []

for X, y in train_loader:
    X_train_list.append(X.numpy())
    y_train_list.append(y.numpy())

X_train = np.vstack(X_train_list)
y_train = np.concatenate(y_train_list)

print(f"Training data for ensemble: {X_train.shape}")

Training data for ensemble: (130812, 7)


In [3]:
ensemble = StackingEnsemble(config.ensemble, input_dim=input_dim)

print("Training stacking ensemble...")
ensemble.fit(X_train, y_train)
print("✓ Ensemble training complete!")

XGBoost not available, falling back to sklearn GradientBoosting
Training stacking ensemble...
Training Stacking Ensemble with 2 base models...


Base models:   0%|          | 0/2 [00:00<?, ?it/s]

  Training mlp...


  Retraining mlp on full data...


Base models:  50%|█████     | 1/2 [20:39<20:39, 1239.28s/it]

  Training xgboost...


  Retraining xgboost on full data...


Base models: 100%|██████████| 2/2 [22:32<00:00, 676.35s/it] 

  Training Meta-Learner...
Ensemble Training Complete.
✓ Ensemble training complete!


In [4]:
X_test_list = []
y_test_list = []

for X, y in test_loader:
    X_test_list.append(X.numpy())
    y_test_list.append(y.numpy())

X_test = np.vstack(X_test_list)
y_test = np.concatenate(y_test_list)

y_pred_probs = ensemble.predict_proba(X_test)

test_metrics = calculate_metrics(y_test, y_pred_probs)

print("Ensemble Model Test Results:")
for metric, value in test_metrics.items():
    print(f"  {metric.upper()}: {value:.4f}")

Ensemble Model Test Results:
  ACCURACY: 0.9886
  PRECISION: 0.9844
  RECALL: 0.9908
  F1: 0.9876
  AUC: 0.9993


In [5]:
import joblib

os.makedirs('../reports/results/checkpoints', exist_ok=True)
joblib.dump(ensemble, '../reports/results/checkpoints/ensemble_model.joblib')

print("✓ Ensemble model saved!")

✓ Ensemble model saved!
